In [ ]:
import nltk

# New Section

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files

In [ ]:
df = pd.read_csv('Questionnaire LS.csv')

In [ ]:
df

In [ ]:
dataframe = df[['Response','Type']]

In [ ]:
dataframe

,Response,Type
0,A book with lots of questions and answers to p...,Visual
1,Trace the letters in the air (finger spelling),Kinaesthetic
2,Make up index cards that you can review,Kinaesthetic
3,create flash cards to remeber a sentence,Kinaesthetic
4,Read and write the poen twice,Read/Write
...,...,...
651,Books and website refer,Visual
652,Paper Presentation,Visual
653,handouts or a text book,Read/Write
654,By Viewing information in word form,Read/Write


In [ ]:
dataframe.describe()

,Response,Type
count,656,656
unique,56,4
top,Books and website refer,Visual
freq,28,183


In [ ]:
dataframe.groupby('Type').describe()

Response                                                    
                count unique                                     top freq
Type                                                                     
Aural             120     18  Ask a friend to show you on a computer   18
Kinaesthetic      171     21                               Hackathon   18
Read/Write        182     22           Read and write the poen twice   18
Visual            183     19                 Books and website refer   26

In [ ]:
dataframe['length']= dataframe['Response'].apply(len)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
dataframe

,Response,Type,length
0,A book with lots of questions and answers to p...,Visual,53
1,Trace the letters in the air (finger spelling),Kinaesthetic,46
2,Make up index cards that you can review,Kinaesthetic,39
3,create flash cards to remeber a sentence,Kinaesthetic,40
4,Read and write the poen twice,Read/Write,29
...,...,...,...
651,Books and website refer,Visual,23
652,Paper Presentation,Visual,18
653,handouts or a text book,Read/Write,23
654,By Viewing information in word form,Read/Write,35


In [ ]:
dataframe.describe()

,length
count,656.000000
mean,38.265244
std,14.641354
min,9.000000
25%,29.000000
50%,38.000000
75%,45.000000
max,72.000000


In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def text_process(mess):
  nopunc = [char for char in mess if char not in string.punctuation]
  nopunc = ''.join(nopunc)
  return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]



In [ ]:
dataframe

,Response,Type,length
0,A book with lots of questions and answers to p...,Visual,53
1,Trace the letters in the air (finger spelling),Kinaesthetic,46
2,Make up index cards that you can review,Kinaesthetic,39
3,create flash cards to remeber a sentence,Kinaesthetic,40
4,Read and write the poen twice,Read/Write,29
...,...,...,...
651,Books and website refer,Visual,23
652,Paper Presentation,Visual,18
653,handouts or a text book,Read/Write,23
654,By Viewing information in word form,Read/Write,35


In [ ]:
dataframe['Response'].head(5).apply(text_process)

0    [book, lots, questions, answers, practice]
1       [Trace, letters, air, finger, spelling]
2                  [Make, index, cards, review]
3     [create, flash, cards, remeber, sentence]
4                    [Read, write, poen, twice]
Name: Response, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(dataframe['Response'])

In [ ]:
print(len(bow_transformer.vocabulary_))

158


In [ ]:
dataframe['Response']

0      A book with lots of questions and answers to p...
1         Trace the letters in the air (finger spelling)
2                Make up index cards that you can review
3               create flash cards to remeber a sentence
4                          Read and write the poen twice
                             ...                        
651                              Books and website refer
652                                   Paper Presentation
653                              handouts or a text book
654                  By Viewing information in word form
655                        Visit the websites and learn 
Name: Response, Length: 656, dtype: object

In [ ]:
message= dataframe['Response'][3]
print(message)

create flash cards to remeber a sentence


In [ ]:
bow4 = bow_transformer.transform([message])
print(bow4)
print(bow4.shape)

  (0, 47)	1
  (0, 56)	1
  (0, 68)	1
  (0, 119)	1
  (0, 129)	1
(1, 158)


In [ ]:
messages_bow = bow_transformer.transform(dataframe['Response'])
print('Shape of Sparse Matrix: ', messages_bow.shape)
print('Amount of Non-Zero occurences: ', messages_bow.nnz)

Shape of Sparse Matrix:  (656, 158)
Amount of Non-Zero occurences:  2684


In [ ]:
sparsity = (100.0 * messages_bow.nnz / (messages_bow.shape[0] * messages_bow.shape[1]))
print('sparsity: {}'.format(round(sparsity)))

sparsity: 3


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(messages_bow)
tfidf4 = tfidf_transformer.transform(bow4)

In [ ]:
messages_tfidf = tfidf_transformer.transform(messages_bow)
print(messages_tfidf.shape)

(656, 158)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
text_classification_model = MultinomialNB().fit(messages_tfidf, dataframe['Type'])

In [ ]:
print('predicted:', text_classification_model.predict(tfidf4)[0])
print('expected:', dataframe.Type[3])

predicted: Kinaesthetic
expected: Kinaesthetic


In [ ]:
all_predictions = text_classification_model.predict(messages_tfidf)

In [ ]:
print(all_predictions)

['Visual' 'Kinaesthetic' 'Kinaesthetic' 'Kinaesthetic' 'Read/Write'
 'Read/Write' 'Read/Write' 'Visual' 'Aural' 'Read/Write' 'Visual' 'Visual'
 'Kinaesthetic' 'Kinaesthetic' 'Visual' 'Aural' 'Visual' 'Kinaesthetic'
 'Kinaesthetic' 'Kinaesthetic' 'Read/Write' 'Read/Write' 'Read/Write'
 'Visual' 'Aural' 'Read/Write' 'Visual' 'Visual' 'Kinaesthetic'
 'Kinaesthetic' 'Visual' 'Aural' 'Visual' 'Kinaesthetic' 'Kinaesthetic'
 'Kinaesthetic' 'Read/Write' 'Read/Write' 'Read/Write' 'Visual' 'Aural'
 'Read/Write' 'Visual' 'Visual' 'Kinaesthetic' 'Kinaesthetic' 'Visual'
 'Aural' 'Visual' 'Kinaesthetic' 'Kinaesthetic' 'Aural' 'Read/Write'
 'Read/Write' 'Read/Write' 'Visual' 'Aural' 'Read/Write' 'Visual' 'Visual'
 'Kinaesthetic' 'Kinaesthetic' 'Visual' 'Aural' 'Visual' 'Aural'
 'Kinaesthetic' 'Aural' 'Read/Write' 'Read/Write' 'Read/Write' 'Aural'
 'Aural' 'Read/Write' 'Visual' 'Visual' 'Kinaesthetic' 'Kinaesthetic'
 'Visual' 'Aural' 'Visual' 'Aural' 'Kinaesthetic' 'Aural' 'Read/Write'
 'Read/Write' '

In [ ]:
from sklearn.metrics import classification_report
print (classification_report(dataframe['Type'], all_predictions))

              precision    recall  f1-score   support

       Aural       0.92      0.90      0.91       120
Kinaesthetic       0.95      0.92      0.93       171
  Read/Write       0.94      0.93      0.93       182
      Visual       0.91      0.95      0.93       183

    accuracy                           0.93       656
   macro avg       0.93      0.92      0.93       656
weighted avg       0.93      0.93      0.93       656



In [ ]:
##prediction Model Save and load

In [ ]:
import pickle

In [ ]:
pickle.dump(text_classification_model, open('model.pkl', 'wb'))

In [ ]:
pickled_model = pickle.load(open('model.pkl', 'rb'))

##pickled_model.predict('Hi How are you')

In [ ]:
#Testing the sampel message
test_message= [['tiny'],
               ['relaxed'],
               ['play'],['goes']]

In [ ]:
#Making  the dataframe
df = pd.DataFrame(test_message, columns=["Test_Message"])


In [ ]:
#viewing the dataframe
df

,Test_Message
0,tiny
1,relaxed
2,play
3,goes


In [ ]:
#Applying the transfomer
messages_bow = bow_transformer.transform(df['Test_Message'])
bow_transformer1 = CountVectorizer(analyzer=text_process).fit(dataframe['Response'])

In [ ]:
messages_bow

<4x158 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [ ]:
#Applying the TfidTransformer for converting the words to sequence
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(messages_bow)
messages_tfidf = tfidf_transformer.transform(messages_bow)

In [ ]:
print('predicted:', pickled_model.predict(messages_tfidf))


predicted: ['Visual' 'Visual' 'Kinaesthetic' 'Visual']


In [ ]:
pickled_model.predict_proba(messages_tfidf)

array([[0.18292683, 0.26067073, 0.27743902, 0.27896341],
       [0.18292683, 0.26067073, 0.27743902, 0.27896341],
       [0.09613545, 0.61242687, 0.1699144 , 0.12152328],
       [0.18292683, 0.26067073, 0.27743902, 0.27896341]])

In [ ]:
result=pickled_model.predict(messages_tfidf)